In [ ]:
#Configure git user name and user e-mail
!git config global --user.name "name goes here"
!git config global --user.emil "email goes here"
#Clone the repo into /content files
!git clone "repo goes here"
# Change the directory to lick-caption-bias
%cd mlrc-lic-plus-age-fork
!git checkout visualize-bert

fatal: not in a git directory
fatal: not in a git directory
Cloning into 'mlrc-lic-plus-age-fork'...
remote: Enumerating objects: 578, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 578 (delta 140), reused 188 (delta 107), pack-reused 328
Receiving objects: 100% (578/578), 21.99 MiB | 24.97 MiB/s, done.
Resolving deltas: 100% (324/324), done.
/content/mlrc-lic-plus-age-fork
Error downloading object: saved_models/age_annotation_generated_model_lstm_nic_seed_0_epoch_0.pt (a5ab778): Smudge error: Error downloading saved_models/age_annotation_generated_model_lstm_nic_seed_0_epoch_0.pt (a5ab7782732eff0705c43928b99ab052dd75fc2dbfc155cb19c49e5f15bdf64f): batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.

Errors logged to /content/mlrc-lic-plus-age-fork/.git/lfs/logs/20230126T081254.282228447.log
Use `git lfs logs last` to view t

In [ ]:
!pip install jupyterlab
!pip install ipywidgets
!pip install bertviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.0/424.0 KB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 108.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.2/365.2 KB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/weight_bert_nic.zip -d /content/mlrc-lic-plus-age-fork/saved_models

Archive:  /content/drive/MyDrive/weight_bert_nic.zip
  inflating: /content/mlrc-lic-plus-age-fork/saved_models/content/mlrc-lic-plus-age-fork/saved_models/age_annotation_generated_model_bert_nic_seed_456_epoch_4.pt  


In [ ]:
from transformers import BertTokenizer
from transformers import PYTORCH_PRETRAINED_BERT_CACHE
from transformers import BertConfig, WEIGHTS_NAME, CONFIG_NAME
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertModel
from transformers import BertPreTrainedModel
from bertviz import head_view
import torch
# import age variables from utils
from age_utils import (
  young_words,
  old_words,
  age_words
)
# import age functiond from utils
from age_utils import (
  gender_pickle_generator,
  race_pickle_generator,
  label_human_caption,
  label_human_annotations,
  match_labels,
  make_train_test_split,

)
from age_dataset import BERT_ANN_leak_data, BERT_MODEL_leak_data
from collections import namedtuple
import nltk
from model import BERT_GenderClassifier
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def load_bert_model(file_name, model):
  '''
  Load the model from the weight file.
  Please note that the weights will also contain the classification head.
  So initialize the model accordingly.
  Arguments
  ---------
  file_name : str
      path to the weights
  model : torch.nn.Module
      BERT Gender_Classifier Model
  '''
  model.load_state_dict(torch.load(file_name))

def find_top_5(model,data):
  '''
  This function find the most confident predictions from the bert classifier.
  '''

  model.eval()
  correct_classified = []
  wrong_classified = []
  for index in range(len(data)):
    input_ids, attention_mask, token_type_ids, age_target, img_id = data[index]
    # Convert everything to gpu.
    input_ids = torch.unsqueeze(input_ids.cuda(), dim = 0)
    attention_mask = torch.unsqueeze(attention_mask.cuda(), dim = 0)
    token_type_ids = torch.unsqueeze(token_type_ids.cuda(), dim = 0)
    age_target = torch.squeeze(age_target).cuda()
    outputs = model.lang_model(input_ids, 
                  attention_mask=attention_mask, 
                  token_type_ids=token_type_ids, output_hidden_states = True, output_attentions = True)
    probs = model.classifier(outputs.last_hidden_state[:, 0, :])
    probs = torch.softmax(probs, dim = 1)
    if age_target == torch.argmax(probs):
      correct_classified.append((index, probs))
    else:
      wrong_classified.append((index,probs))
  return correct_classified, wrong_classified
def visualize_attention(model, data, tokenizer, index):
  '''
  Visualize the attention from the bert model.
  Arguments
  ---------
  model : torch.nn.Module
      BERT Model
  data : torch.nn.Dataset
      Annotation data from authors paper
  tokenizer : BertTokenizer
      Uncased bert tokenizer for input parsing
  index : int
      Index of datapoint to visualize
  
  Returns
  -------
  IPYWidget that displays attention matrix.
  
  '''
  # Get outputs from the dataloader.
  model.eval()
  input_ids, attention_mask, token_type_ids, age_target, img_id = data[index]
  # Convert everything to gpu.
  input_ids = torch.unsqueeze(input_ids.cuda(), dim = 0)
  attention_mask = torch.unsqueeze(attention_mask.cuda(), dim = 0)
  token_type_ids = torch.unsqueeze(token_type_ids.cuda(), dim = 0)
  age_target = torch.squeeze(age_target).cuda()
  # Pass the arguments to model
  outputs = model.lang_model(input_ids, 
                  attention_mask=attention_mask, 
                  token_type_ids=token_type_ids, output_hidden_states = True, output_attentions = True)

  # Calculate the probabilities from the model 
  probs = model.classifier(outputs.last_hidden_state[:, 0, :])
  map = {0 : 'Young', 1 : 'Old'}
  probs = torch.softmax(probs, dim = 1)
  print(age_target, probs)
  print(f"Target is : {map[age_target.item()]}, image_id : {img_id} \n Predicted : \n Young: {probs[0,0].item()} \n Old: {probs[0,1].item()} ")
  # Get the attention matrices.
  attention = outputs[-1]    
  # Convert tokens into words so it is readable
  tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
  # Find the index of [SEP] token. This indicates that sentence has finished.
  first_sep = list(tokens).index('[SEP]') 
  # Get rid of padding attentions and tokens.
  new_attention = torch.zeros((len(attention), 1, len(attention), first_sep + 1, first_sep + 1))
  for id,att in enumerate(attention):
    new_attention[id] = att[:, :,  :first_sep + 1, :first_sep + 1]
  # Return the widget.
  return head_view(new_attention, tokens[:first_sep + 1], html_action = 'return')  # Display model view

In [ ]:
# Emulete fake args.
ARG = namedtuple('args', ['hidden_dim', 'batch_size', 'workers','test_ratio','task', 'align_vocab','max_seq_length', 'cap_model', 'freeze_bert'])
args = ARG(256, 64,1,0.1, 'captioning', True, 64, 'nic', True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BERT_GenderClassifier(args, tokenizer).cuda()
# Load the bert model
load_bert_model('/content/mlrc-lic-plus-age-fork/saved_models/content/mlrc-lic-plus-age-fork/saved_models/age_annotation_generated_model_bert_nic_seed_456_epoch_4.pt', model)

# Define the human annotation entries using age_utils.
age_val_obj_cap_entries = label_human_annotations(gender_pickle_generator('human'),young_words,old_words) # Human captions
selected_cap_age_entries = match_labels(age_val_obj_cap_entries,gender_pickle_generator(args.cap_model))

d_train, d_test = make_train_test_split(args, selected_cap_age_entries)

# Define the dataset from authors code
trainANNCAPobject = BERT_MODEL_leak_data(d_train, d_test, args,  selected_cap_age_entries, age_words, tokenizer,
                                                args.max_seq_length, split='train', caption_ind=0)
testANNCAPobject = BERT_MODEL_leak_data(d_train, d_test, args,  selected_cap_age_entries, age_words, tokenizer,
                                                args.max_seq_length, split='test', caption_ind=0)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


***Freeze BERT***
#train : #test =  7260 806


In [ ]:
correct, wrong = find_top_5(model, testANNCAPobject)

In [ ]:
correct = sorted(correct, key=lambda student: torch.max(student[1]), reverse = True)
wrong  = sorted(wrong, key=lambda student: torch.max(student[1]), reverse = True)

In [ ]:
correct_old = list(filter(lambda x: torch.argmax(x[1]) == 1, correct))

In [ ]:
visualize_attention(model,testANNCAPobject,tokenizer, correct[41][0])

tensor(0, device='cuda:0') tensor([[0.9875, 0.0125]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Target is : Young, image_id : 446743 
 Predicted : 
 Young: 0.9875141978263855 
 Old: 0.012485825456678867 
